In [1]:
#dependencies and load data
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree

crashes_df = pd.read_csv("Resources\crashes_cleaned_df.csv")
crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56415 entries, 0 to 56414
Columns: 160 entries, Unnamed: 0 to STAT_DIV_NAME_Metro
dtypes: float64(2), int64(157), object(1)
memory usage: 68.9+ MB


In [2]:
# Generate summary statistics
crashes_df = crashes_df.drop(columns=["ACCIDENT_DATE"])
crashes_df.describe()

,Unnamed: 0,ACCIDENT_TIME,LONGITUDE,LATITUDE,TOTAL_PERSONS,INJ_OR_FATAL,FATALITY,SERIOUSINJURY,OTHERINJURY,NONINJURED,...,"RMA_ALL_Arterial Other,Arterial Highway","RMA_ALL_Arterial Other,Local Road",RMA_ALL_Freeway,"RMA_ALL_Freeway,Arterial Other",RMA_ALL_Local Road,"RMA_ALL_Local Road,Arterial Highway","RMA_ALL_Local Road,Arterial Other",RMA_ALL_Other,STAT_DIV_NAME_Country,STAT_DIV_NAME_Metro
count,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,...,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000,56415.000000
mean,30299.335070,13.240433,144.743488,-37.659963,2.399202,1.320642,0.021235,0.467252,0.832155,1.025880,...,0.013986,0.134822,0.064912,0.007055,0.355083,0.048870,0.026234,0.014978,0.292015,0.707985
std,17515.740927,5.115060,5.785175,1.486331,1.505197,0.781959,0.152418,0.660554,0.793266,1.273791,...,0.117432,0.341537,0.246372,0.083697,0.478543,0.215598,0.159832,0.121467,0.454693,0.454693
min,0.000000,0.000000,-1.000000,-39.023993,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15102.500000,9.000000,144.840401,-37.957271,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,30313.000000,14.000000,145.013377,-37.816115,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,45432.500000,17.000000,145.205104,-37.695981,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,60688.000000,23.000000,149.757513,-1.000000,89.000000,27.000000,4.000000,16.000000,25.000000,87.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
columns_list = crashes_df.columns.to_list()
print(columns_list)

['Unnamed: 0', 'ACCIDENT_TIME', 'LONGITUDE', 'LATITUDE', 'TOTAL_PERSONS', 'INJ_OR_FATAL', 'FATALITY', 'SERIOUSINJURY', 'OTHERINJURY', 'NONINJURED', 'MALES', 'FEMALES', 'BICYCLIST', 'PASSENGER', 'DRIVER', 'PEDESTRIAN', 'PILLION', 'MOTORIST', 'PED_CYCLIST_5_12', 'PED_CYCLIST_13_18', 'OLD_PEDESTRIAN', 'OLD_DRIVER', 'YOUNG_DRIVER', 'UNLICENCSED', 'NO_OF_VEHICLES', 'HEAVYVEHICLE', 'PASSENGERVEHICLE', 'MOTORCYCLE', 'PUBLICVEHICLE', 'ABS_CODE_ABS to receive accident', 'ABS_CODE_Heart attk/suicide/death by nat.causes', 'ABS_CODE_Non ABS accident', 'ACCIDENT_STATUS_Discarded', 'ACCIDENT_STATUS_Finished', 'ACCIDENT_STATUS_Private Property', 'ACCIDENT_STATUS_Reopened', 'ACCIDENT_STATUS_Unfinished', 'ALCOHOLTIME_No', 'ALCOHOLTIME_Yes', 'ACCIDENT_TYPE_Collision with a fixed object', 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle', 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident', 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYP

In [4]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crashes_scaled = StandardScaler().fit_transform(crashes_df[columns_list])
crashes_scaled[:5]

array([[-1.72984995e+00, -1.61102815e+00, -6.48320087e-02,
        -3.48112673e-01, -2.65218369e-01, -4.10052758e-01,
        -1.39325653e-01, -7.07368974e-01,  2.11589763e-01,
        -2.03171987e-02, -1.32532210e+00,  4.79915495e-02,
        -3.17216886e-01,  4.21473485e-01, -5.56983100e-01,
        -2.91194674e-01, -7.07794945e-02, -3.95172692e-01,
        -8.74029256e-02, -1.21622591e-01, -1.36643499e-01,
        -2.66420406e-01, -5.83347088e-01, -1.88184361e-01,
        -1.09043893e+00, -2.12993956e-01, -5.05128085e-01,
        -3.95561824e-01, -1.12595965e-01,  7.70566936e-02,
        -5.44884800e-02, -5.43246130e-02, -1.57550801e-02,
         7.75208797e-02, -2.69682337e-02, -1.26316069e-02,
        -6.97328449e-02, -1.31453340e+00,  1.31453340e+00,
         2.30657760e+00, -1.34856814e+00, -7.75208797e-02,
        -2.01252098e-01, -2.85666130e-02, -3.07549555e-01,
        -9.96772450e-02, -2.16863010e-01, -9.88577843e-02,
        -3.72650911e-01, -4.00316597e-01, -4.12367903e-0

In [5]:
# Create a DataFrame with the scaled data
crashes_scaled_df=pd.DataFrame(crashes_scaled, columns=columns_list)
crashes_scaled_df.head()

,Unnamed: 0,ACCIDENT_TIME,LONGITUDE,LATITUDE,TOTAL_PERSONS,INJ_OR_FATAL,FATALITY,SERIOUSINJURY,OTHERINJURY,NONINJURED,...,"RMA_ALL_Arterial Other,Arterial Highway","RMA_ALL_Arterial Other,Local Road",RMA_ALL_Freeway,"RMA_ALL_Freeway,Arterial Other",RMA_ALL_Local Road,"RMA_ALL_Local Road,Arterial Highway","RMA_ALL_Local Road,Arterial Other",RMA_ALL_Other,STAT_DIV_NAME_Country,STAT_DIV_NAME_Metro
0,-1.729850,-1.611028,-0.064832,-0.348113,-0.265218,-0.410053,-0.139326,-0.707369,0.211590,-0.020317,...,-0.119097,-0.394756,-0.263473,-0.084291,1.347681,-0.226674,-0.164137,-0.123313,1.557076,-1.557076
1,-1.729793,-0.438011,0.058167,0.002631,-0.929589,-0.410053,-0.139326,-0.707369,0.211590,-0.805382,...,-0.119097,2.533214,-0.263473,-0.084291,-0.742015,-0.226674,-0.164137,-0.123313,-0.642229,0.642229
2,-1.729736,-0.047005,0.026094,-0.104987,-0.265218,-0.410053,-0.139326,-0.707369,0.211590,-0.020317,...,-0.119097,-0.394756,-0.263473,-0.084291,1.347681,-0.226674,-0.164137,-0.123313,-0.642229,0.642229
3,-1.729622,-0.633514,-0.257969,-0.239447,-0.929589,-0.410053,-0.139326,0.806524,-1.049033,-0.805382,...,-0.119097,-0.394756,-0.263473,-0.084291,-0.742015,-0.226674,-0.164137,-0.123313,1.557076,-1.557076
4,-1.729564,-1.024520,0.301377,-0.355901,0.399152,-0.410053,-0.139326,-0.707369,0.211590,0.764748,...,-0.119097,-0.394756,-0.263473,-0.084291,-0.742015,-0.226674,6.092474,-0.123313,1.557076,-1.557076


In [6]:
# Create a list with the number of k-values
k = list(range(1, 87))

In [7]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(crashes_scaled_df)
    inertia.append(k_model.inertia_)

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\evang\anaconda3\envs\dev\lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will c

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 1

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 1

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [8]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [9]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [k]   (inertia)

In [10]:
original_data = pd.read_csv("Resources/Road_Crashes_for_five_Years_Victoria.csv")
original_data = original_data.drop(columns=["X", "Y", "ACCIDENT_NO", "SRNS", "SRNS_ALL", "DIVIDED", "DIVIDED_ALL", "UNKNOWN", "NODE_ID", "OBJECTID"])
original_data = original_data.dropna()

In [11]:
X = crashes_df.drop(columns=['SEVERITY_Fatal accident', 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident', 'SEVERITY_Serious injury accident'])
y = original_data["SEVERITY"]

In [12]:
# Split the dataset using train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Import the KNeighborsClassifier module from sklearn
# Instantiate the KNeighborsClassifier model with n_neighbors = 3 
knn = KNeighborsClassifier(n_neighbors=3)

In [14]:
# Train the model using the training data
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [15]:
# Create predictions using the testing data
y_pred = knn.predict(X_test)

In [16]:
# Print the classification report comparing the testing data to the model 

print(classification_report(y_test, y_pred))

                         precision    recall  f1-score   support

         Fatal accident       0.02      0.02      0.02       303
  Other injury accident       0.61      0.67      0.64      8164
Serious injury accident       0.45      0.38      0.41      5637

               accuracy                           0.54     14104
              macro avg       0.36      0.36      0.36     14104
           weighted avg       0.53      0.54      0.53     14104



In [17]:
from sklearn import tree
treem = tree.DecisionTreeClassifier()

In [18]:
# Fit the model
treem = treem.fit(X_train, y_train)

In [19]:
# Making predictions using the testing data
treem_predictions = treem.predict(X_test)

In [20]:
print(classification_report(y_test, treem_predictions))

                         precision    recall  f1-score   support

         Fatal accident       1.00      1.00      1.00       303
  Other injury accident       1.00      1.00      1.00      8164
Serious injury accident       1.00      1.00      1.00      5637

               accuracy                           1.00     14104
              macro avg       1.00      1.00      1.00     14104
           weighted avg       1.00      1.00      1.00     14104



In [21]:
original_data["SEVERITY"].value_counts()

SEVERITY
Other injury accident      32904
Serious injury accident    22371
Fatal accident              1138
Non injury accident            2
Name: count, dtype: int64

In [22]:
# Create the random forest classifier instance
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [23]:
# Fit the model and use .ravel()on the "y_train" data. 
rf_model = rf_model.fit(X_train, y_train.ravel())

In [24]:
 # Making predictions using the testing data
rf_predictions = rf_model.predict(X_test)

In [25]:
print(classification_report(y_test, rf_predictions))

                         precision    recall  f1-score   support

         Fatal accident       1.00      1.00      1.00       303
  Other injury accident       1.00      1.00      1.00      8164
Serious injury accident       1.00      1.00      1.00      5637

               accuracy                           1.00     14104
              macro avg       1.00      1.00      1.00     14104
           weighted avg       1.00      1.00      1.00     14104



In [26]:
reduced_crashes_df = crashes_df[['DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
                                 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'ACCIDENT_TYPE_Collision with a fixed object',
                                 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle',
                                 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident',
                                 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYPE_Struck animal',
                                 'ACCIDENT_TYPE_Vehicle overturned (no collision)',
                                 'ACCIDENT_TYPE_collision with some other object', 'LIGHT_CONDITION_Dark No street lights',
                                 'LIGHT_CONDITION_Dark Street lights off', 'LIGHT_CONDITION_Dark Street lights on',
                                 'LIGHT_CONDITION_Dark Street lights unknown', 'LIGHT_CONDITION_Day',
                                 'LIGHT_CONDITION_Dusk/Dawn', 'LIGHT_CONDITION_Unk.', 'ROAD_GEOMETRY_Cross intersection',
                                 'ROAD_GEOMETRY_Dead end', 'ROAD_GEOMETRY_Multiple intersection',
                                 'ROAD_GEOMETRY_Not at intersection', 'ROAD_GEOMETRY_Private property',
                                 'ROAD_GEOMETRY_Road closure', 'ROAD_GEOMETRY_T intersection', 'ROAD_GEOMETRY_Unknown',
                                 'ROAD_GEOMETRY_Y intersection','SPEED_ZONE_100 km/hr', 'SPEED_ZONE_110 km/hr',
                                 'SPEED_ZONE_40 km/hr', 'SPEED_ZONE_50 km/hr', 'SPEED_ZONE_60 km/hr', 'SPEED_ZONE_70 km/hr',
                                 'SPEED_ZONE_80 km/hr', 'SPEED_ZONE_90 km/hr', 'SPEED_ZONE_Camping grounds or off road',
                                 'SPEED_ZONE_Not known', 'SPEED_ZONE_Other speed limit', 'TOTAL_PERSONS', 'INJ_OR_FATAL',
                                 'MALES', 'FEMALES', 'UNLICENCSED', 'RMA_Arterial Highway', 'RMA_Arterial Other',
                                 'RMA_Freeway', 'RMA_Local Road', 'RMA_Non Arterial', 'RMA_ALL_Arterial Highway',
                                 'RMA_ALL_Arterial Highway,Arterial Other', 'RMA_ALL_Arterial Highway,Local Road',
                                 'RMA_ALL_Arterial Other', 'RMA_ALL_Arterial Other,Arterial Highway',
                                 'RMA_ALL_Arterial Other,Local Road', 'RMA_ALL_Freeway', 'RMA_ALL_Freeway,Arterial Other',
                                 'RMA_ALL_Local Road', 'RMA_ALL_Local Road,Arterial Highway',
                                 'RMA_ALL_Local Road,Arterial Other', 'RMA_ALL_Other', 'SEVERITY_Fatal accident',
                                 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident',
                                 'SEVERITY_Serious injury accident', 'REGION_NAME_ALL_EASTERN REGION,EASTERN REGION',
                                 'REGION_NAME_ALL_METROPOLITAN NORTH WEST REGION',
                                 'REGION_NAME_ALL_METROPOLITAN SOUTH EAST REGION', 'REGION_NAME_ALL_NORTH EASTERN REGION',
                                 'REGION_NAME_ALL_NORTHERN REGION', 'REGION_NAME_ALL_Other',
                                 'REGION_NAME_ALL_SOUTH WESTERN REGION', 'REGION_NAME_ALL_WESTERN REGION', 
                                 'STAT_DIV_NAME_Country', 'STAT_DIV_NAME_Metro']]
reduced_crashes_df.head()

,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7,ACCIDENT_TYPE_Collision with a fixed object,ACCIDENT_TYPE_Collision with vehicle,ACCIDENT_TYPE_Fall from or in moving vehicle,...,"REGION_NAME_ALL_EASTERN REGION,EASTERN REGION",REGION_NAME_ALL_METROPOLITAN NORTH WEST REGION,REGION_NAME_ALL_METROPOLITAN SOUTH EAST REGION,REGION_NAME_ALL_NORTH EASTERN REGION,REGION_NAME_ALL_NORTHERN REGION,REGION_NAME_ALL_Other,REGION_NAME_ALL_SOUTH WESTERN REGION,REGION_NAME_ALL_WESTERN REGION,STAT_DIV_NAME_Country,STAT_DIV_NAME_Metro
0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
4,0,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


In [27]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
red_scaler = StandardScaler().fit(reduced_crashes_df[['DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
                                 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'ACCIDENT_TYPE_Collision with a fixed object',
                                 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle',
                                 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident',
                                 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYPE_Struck animal',
                                 'ACCIDENT_TYPE_Vehicle overturned (no collision)',
                                 'ACCIDENT_TYPE_collision with some other object', 'LIGHT_CONDITION_Dark No street lights',
                                 'LIGHT_CONDITION_Dark Street lights off', 'LIGHT_CONDITION_Dark Street lights on',
                                 'LIGHT_CONDITION_Dark Street lights unknown', 'LIGHT_CONDITION_Day',
                                 'LIGHT_CONDITION_Dusk/Dawn', 'LIGHT_CONDITION_Unk.', 'ROAD_GEOMETRY_Cross intersection',
                                 'ROAD_GEOMETRY_Dead end', 'ROAD_GEOMETRY_Multiple intersection',
                                 'ROAD_GEOMETRY_Not at intersection', 'ROAD_GEOMETRY_Private property',
                                 'ROAD_GEOMETRY_Road closure', 'ROAD_GEOMETRY_T intersection', 'ROAD_GEOMETRY_Unknown',
                                 'ROAD_GEOMETRY_Y intersection','SPEED_ZONE_100 km/hr', 'SPEED_ZONE_110 km/hr',
                                 'SPEED_ZONE_40 km/hr', 'SPEED_ZONE_50 km/hr', 'SPEED_ZONE_60 km/hr', 'SPEED_ZONE_70 km/hr',
                                 'SPEED_ZONE_80 km/hr', 'SPEED_ZONE_90 km/hr', 'SPEED_ZONE_Camping grounds or off road',
                                 'SPEED_ZONE_Not known', 'SPEED_ZONE_Other speed limit', 'TOTAL_PERSONS', 'INJ_OR_FATAL',
                                 'MALES', 'FEMALES', 'UNLICENCSED', 'RMA_Arterial Highway', 'RMA_Arterial Other',
                                 'RMA_Freeway', 'RMA_Local Road', 'RMA_Non Arterial', 'RMA_ALL_Arterial Highway',
                                 'RMA_ALL_Arterial Highway,Arterial Other', 'RMA_ALL_Arterial Highway,Local Road',
                                 'RMA_ALL_Arterial Other', 'RMA_ALL_Arterial Other,Arterial Highway',
                                 'RMA_ALL_Arterial Other,Local Road', 'RMA_ALL_Freeway', 'RMA_ALL_Freeway,Arterial Other',
                                 'RMA_ALL_Local Road', 'RMA_ALL_Local Road,Arterial Highway',
                                 'RMA_ALL_Local Road,Arterial Other', 'RMA_ALL_Other', 'SEVERITY_Fatal accident',
                                 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident',
                                 'SEVERITY_Serious injury accident', 'REGION_NAME_ALL_EASTERN REGION,EASTERN REGION',
                                 'REGION_NAME_ALL_METROPOLITAN NORTH WEST REGION',
                                 'REGION_NAME_ALL_METROPOLITAN SOUTH EAST REGION', 'REGION_NAME_ALL_NORTH EASTERN REGION',
                                 'REGION_NAME_ALL_NORTHERN REGION', 'REGION_NAME_ALL_Other',
                                 'REGION_NAME_ALL_SOUTH WESTERN REGION', 'REGION_NAME_ALL_WESTERN REGION', 
                                 'STAT_DIV_NAME_Country', 'STAT_DIV_NAME_Metro']])
reduced_crashes_scaled = StandardScaler().fit_transform(reduced_crashes_df[['DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
                                 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'ACCIDENT_TYPE_Collision with a fixed object',
                                 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle',
                                 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident',
                                 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYPE_Struck animal',
                                 'ACCIDENT_TYPE_Vehicle overturned (no collision)',
                                 'ACCIDENT_TYPE_collision with some other object', 'LIGHT_CONDITION_Dark No street lights',
                                 'LIGHT_CONDITION_Dark Street lights off', 'LIGHT_CONDITION_Dark Street lights on',
                                 'LIGHT_CONDITION_Dark Street lights unknown', 'LIGHT_CONDITION_Day',
                                 'LIGHT_CONDITION_Dusk/Dawn', 'LIGHT_CONDITION_Unk.', 'ROAD_GEOMETRY_Cross intersection',
                                 'ROAD_GEOMETRY_Dead end', 'ROAD_GEOMETRY_Multiple intersection',
                                 'ROAD_GEOMETRY_Not at intersection', 'ROAD_GEOMETRY_Private property',
                                 'ROAD_GEOMETRY_Road closure', 'ROAD_GEOMETRY_T intersection', 'ROAD_GEOMETRY_Unknown',
                                 'ROAD_GEOMETRY_Y intersection','SPEED_ZONE_100 km/hr', 'SPEED_ZONE_110 km/hr',
                                 'SPEED_ZONE_40 km/hr', 'SPEED_ZONE_50 km/hr', 'SPEED_ZONE_60 km/hr', 'SPEED_ZONE_70 km/hr',
                                 'SPEED_ZONE_80 km/hr', 'SPEED_ZONE_90 km/hr', 'SPEED_ZONE_Camping grounds or off road',
                                 'SPEED_ZONE_Not known', 'SPEED_ZONE_Other speed limit', 'TOTAL_PERSONS', 'INJ_OR_FATAL',
                                 'MALES', 'FEMALES', 'UNLICENCSED', 'RMA_Arterial Highway', 'RMA_Arterial Other',
                                 'RMA_Freeway', 'RMA_Local Road', 'RMA_Non Arterial', 'RMA_ALL_Arterial Highway',
                                 'RMA_ALL_Arterial Highway,Arterial Other', 'RMA_ALL_Arterial Highway,Local Road',
                                 'RMA_ALL_Arterial Other', 'RMA_ALL_Arterial Other,Arterial Highway',
                                 'RMA_ALL_Arterial Other,Local Road', 'RMA_ALL_Freeway', 'RMA_ALL_Freeway,Arterial Other',
                                 'RMA_ALL_Local Road', 'RMA_ALL_Local Road,Arterial Highway',
                                 'RMA_ALL_Local Road,Arterial Other', 'RMA_ALL_Other', 'SEVERITY_Fatal accident',
                                 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident',
                                 'SEVERITY_Serious injury accident', 'REGION_NAME_ALL_EASTERN REGION,EASTERN REGION',
                                 'REGION_NAME_ALL_METROPOLITAN NORTH WEST REGION',
                                 'REGION_NAME_ALL_METROPOLITAN SOUTH EAST REGION', 'REGION_NAME_ALL_NORTH EASTERN REGION',
                                 'REGION_NAME_ALL_NORTHERN REGION', 'REGION_NAME_ALL_Other',
                                 'REGION_NAME_ALL_SOUTH WESTERN REGION', 'REGION_NAME_ALL_WESTERN REGION', 
                                 'STAT_DIV_NAME_Country', 'STAT_DIV_NAME_Metro']])
reduced_crashes_scaled[:5]

array([[-3.72650911e-01, -4.00316597e-01, -4.12367903e-01,
         2.38916002e+00, -4.20981581e-01, -4.34840293e-01,
        -3.96702931e-01,  2.30657760e+00, -1.34856814e+00,
        -7.75208797e-02, -2.01252098e-01, -2.85666130e-02,
        -3.07549555e-01, -9.96772450e-02, -2.16863010e-01,
        -9.88577843e-02, -2.44285529e-01, -4.82439847e-02,
         2.29847638e+00, -9.98584698e-02, -1.44910165e+00,
        -2.71500384e-01, -1.64250781e-01, -5.38367438e-01,
        -3.54981186e-02, -1.45273722e-01,  9.59773229e-01,
        -1.45861112e-02, -5.95422717e-03, -5.40503320e-01,
        -6.65829336e-02, -4.57823812e-02, -4.18703356e-01,
        -1.07542353e-01, -2.57223568e-01, -4.38121145e-01,
         1.40490192e+00, -2.53323782e-01, -4.19171089e-01,
        -5.14600393e-02, -9.46582741e-02, -2.42446848e-01,
        -5.70471273e-02, -2.65218369e-01, -4.10052758e-01,
        -1.32532210e+00,  4.79915495e-02, -1.88184361e-01,
        -4.89646682e-01, -7.68264287e-01, -2.94983328e-0

In [28]:
red_X = pd.DataFrame(reduced_crashes_scaled, columns=[['DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
                                 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'ACCIDENT_TYPE_Collision with a fixed object',
                                 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle',
                                 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident',
                                 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYPE_Struck animal',
                                 'ACCIDENT_TYPE_Vehicle overturned (no collision)',
                                 'ACCIDENT_TYPE_collision with some other object', 'LIGHT_CONDITION_Dark No street lights',
                                 'LIGHT_CONDITION_Dark Street lights off', 'LIGHT_CONDITION_Dark Street lights on',
                                 'LIGHT_CONDITION_Dark Street lights unknown', 'LIGHT_CONDITION_Day',
                                 'LIGHT_CONDITION_Dusk/Dawn', 'LIGHT_CONDITION_Unk.', 'ROAD_GEOMETRY_Cross intersection',
                                 'ROAD_GEOMETRY_Dead end', 'ROAD_GEOMETRY_Multiple intersection',
                                 'ROAD_GEOMETRY_Not at intersection', 'ROAD_GEOMETRY_Private property',
                                 'ROAD_GEOMETRY_Road closure', 'ROAD_GEOMETRY_T intersection', 'ROAD_GEOMETRY_Unknown',
                                 'ROAD_GEOMETRY_Y intersection','SPEED_ZONE_100 km/hr', 'SPEED_ZONE_110 km/hr',
                                 'SPEED_ZONE_40 km/hr', 'SPEED_ZONE_50 km/hr', 'SPEED_ZONE_60 km/hr', 'SPEED_ZONE_70 km/hr',
                                 'SPEED_ZONE_80 km/hr', 'SPEED_ZONE_90 km/hr', 'SPEED_ZONE_Camping grounds or off road',
                                 'SPEED_ZONE_Not known', 'SPEED_ZONE_Other speed limit', 'TOTAL_PERSONS', 'INJ_OR_FATAL',
                                 'MALES', 'FEMALES', 'UNLICENCSED', 'RMA_Arterial Highway', 'RMA_Arterial Other',
                                 'RMA_Freeway', 'RMA_Local Road', 'RMA_Non Arterial', 'RMA_ALL_Arterial Highway',
                                 'RMA_ALL_Arterial Highway,Arterial Other', 'RMA_ALL_Arterial Highway,Local Road',
                                 'RMA_ALL_Arterial Other', 'RMA_ALL_Arterial Other,Arterial Highway',
                                 'RMA_ALL_Arterial Other,Local Road', 'RMA_ALL_Freeway', 'RMA_ALL_Freeway,Arterial Other',
                                 'RMA_ALL_Local Road', 'RMA_ALL_Local Road,Arterial Highway',
                                 'RMA_ALL_Local Road,Arterial Other', 'RMA_ALL_Other', 'SEVERITY_Fatal accident',
                                 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident',
                                 'SEVERITY_Serious injury accident', 'REGION_NAME_ALL_EASTERN REGION,EASTERN REGION',
                                 'REGION_NAME_ALL_METROPOLITAN NORTH WEST REGION',
                                 'REGION_NAME_ALL_METROPOLITAN SOUTH EAST REGION', 'REGION_NAME_ALL_NORTH EASTERN REGION',
                                 'REGION_NAME_ALL_NORTHERN REGION', 'REGION_NAME_ALL_Other',
                                 'REGION_NAME_ALL_SOUTH WESTERN REGION', 'REGION_NAME_ALL_WESTERN REGION', 
                                 'STAT_DIV_NAME_Country', 'STAT_DIV_NAME_Metro']])
red_X.head()

,DAY_OF_WEEK_1,DAY_OF_WEEK_2,DAY_OF_WEEK_3,DAY_OF_WEEK_4,DAY_OF_WEEK_5,DAY_OF_WEEK_6,DAY_OF_WEEK_7,ACCIDENT_TYPE_Collision with a fixed object,ACCIDENT_TYPE_Collision with vehicle,ACCIDENT_TYPE_Fall from or in moving vehicle,...,"REGION_NAME_ALL_EASTERN REGION,EASTERN REGION",REGION_NAME_ALL_METROPOLITAN NORTH WEST REGION,REGION_NAME_ALL_METROPOLITAN SOUTH EAST REGION,REGION_NAME_ALL_NORTH EASTERN REGION,REGION_NAME_ALL_NORTHERN REGION,REGION_NAME_ALL_Other,REGION_NAME_ALL_SOUTH WESTERN REGION,REGION_NAME_ALL_WESTERN REGION,STAT_DIV_NAME_Country,STAT_DIV_NAME_Metro
0,-0.372651,-0.400317,-0.412368,2.389160,-0.420982,-0.43484,-0.396703,2.306578,-1.348568,-0.077521,...,-0.242201,-0.734679,-0.740121,-0.238446,-0.238985,-0.064676,3.474789,-0.23338,1.557076,-1.557076
1,-0.372651,-0.400317,-0.412368,2.389160,-0.420982,-0.43484,-0.396703,2.306578,-1.348568,-0.077521,...,-0.242201,1.361139,-0.740121,-0.238446,-0.238985,-0.064676,-0.287787,-0.23338,-0.642229,0.642229
2,-0.372651,-0.400317,-0.412368,2.389160,-0.420982,-0.43484,-0.396703,-0.433543,0.741527,-0.077521,...,-0.242201,1.361139,-0.740121,-0.238446,-0.238985,-0.064676,-0.287787,-0.23338,-0.642229,0.642229
3,-0.372651,-0.400317,-0.412368,-0.418557,2.375401,-0.43484,-0.396703,-0.433543,-1.348568,-0.077521,...,-0.242201,-0.734679,-0.740121,-0.238446,-0.238985,-0.064676,3.474789,-0.23338,1.557076,-1.557076
4,-0.372651,-0.400317,-0.412368,-0.418557,2.375401,-0.43484,-0.396703,-0.433543,0.741527,-0.077521,...,4.128804,-0.734679,-0.740121,-0.238446,-0.238985,-0.064676,-0.287787,-0.23338,1.557076,-1.557076


In [29]:
values = original_data['LGA_NAME'].value_counts()
print(values[values < 500])

LGA_NAME
MILDURA               474
MACEDON RANGES        438
MURRINDINDI           421
COLAC OTWAY           387
MOORABOOL             385
SOUTH GIPPSLAND       372
SURF COAST            367
CAMPASPE              364
BASS COAST            341
MOIRA                 289
WANGARATTA            287
GOLDEN PLAINS         285
WARRNAMBOOL           252
WODONGA               228
CORANGAMITE           221
MANSFIELD             204
MOYNE                 200
ALPINE                195
HEPBURN               191
GLENELG               186
STRATHBOGIE           180
SWAN HILL             174
HORSHAM               172
INDIGO                171
BENALLA               167
NORTHERN GRAMPIANS    161
MOUNT ALEXANDER       158
SOUTHERN GRAMPIANS    141
TOWONG                134
PYRENEES              127
ARARAT                126
CENTRAL GOLDFIELDS    111
LODDON                105
GANNAWARRA            102
BULOKE                 67
HINDMARSH              49
WEST WIMMERA           46
YARRIAMBIACK           43
(MO

In [30]:
LGA_to_replace = ["MILDURA",
"MACEDON RANGES",
"MURRINDINDI",
"COLAC OTWAY",
"MOORABOOL",
"SOUTH GIPPSLAND",
"SURF COAST",
"CAMPASPE",
"BASS COAST",
"MOIRA",
"WANGARATTA",
"GOLDEN PLAINS",
"WARRNAMBOOL",
"WODONGA",
"CORANGAMITE",
"MANSFIELD",
"MOYNE",
"ALPINE",
"HEPBURN",
"GLENELG",
"STRATHBOGIE",
"SWAN HILL",
"HORSHAM",
"INDIGO",
"BENALLA",
"NORTHERN GRAMPIANS",
"MOUNT ALEXANDER",
"SOUTHERN GRAMPIANS",
"TOWONG",
"PYRENEES",
"ARARAT",
"CENTRAL GOLDFIELDS",
"LODDON",
"GANNAWARRA",
"BULOKE",
"HINDMARSH",
"WEST WIMMERA",
"YARRIAMBIACK",
"(MOUNT HOTHAM)",
"QUEENSCLIFFE",
"(LAKE MOUNTAIN)",
"(MOUNT BULLER)",
"(FALLS CREEK)",
"(MOUNT BAW BAW)",
"(FRENCH ISLAND)",
"",
"(MOUNT STIRLING)"]

for lga in LGA_to_replace:
    original_data['LGA_NAME'] = original_data['LGA_NAME'].replace(lga,"Other")

# Check to make sure binning was successful
original_data['LGA_NAME'].value_counts()

LGA_NAME
Other                   8383
MELBOURNE               3049
CASEY                   2420
GEELONG                 2273
DANDENONG               1936
HUME                    1870
BRIMBANK                1683
MONASH                  1663
WHITTLESEA              1656
MORELAND                1619
YARRA RANGES            1582
YARRA                   1395
DAREBIN                 1377
KINGSTON                1355
WYNDHAM                 1270
BOROONDARA              1231
WHITEHORSE              1208
MORNINGTON PENINSULA    1163
STONNINGTON             1162
KNOX                    1134
PORT PHILLIP            1103
GLEN EIRA               1083
BENDIGO                 1077
BALLARAT                1040
FRANKSTON               1004
MELTON                   955
MOONEE VALLEY            905
CARDINIA                 882
MAROONDAH                851
BANYULE                  841
MARIBYRNONG              792
HOBSONS BAY              770
MANNINGHAM               732
SHEPPARTON               730
BAYSI

In [43]:
red_y = original_data['REGION_NAME_ALL']

In [44]:
red_X_train, red_X_test, red_y_train, red_y_test = train_test_split(red_X, red_y, random_state=42)

In [45]:
red_treem = tree.DecisionTreeClassifier()

In [46]:
red_treem = red_treem.fit(red_X_train, red_y_train.ravel())

In [47]:
red_treem_predictions = red_treem.predict(red_X_test)

In [48]:
print(classification_report(red_y_test, red_treem_predictions))

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\evang\anaconda3\envs\dev\li

                                                               precision    recall  f1-score   support

                                EASTERN REGION,EASTERN REGION       1.00      1.00      1.00       769
 EASTERN REGION,EASTERN REGION,METROPOLITAN SOUTH EAST REGION       0.50      0.67      0.57         3
 EASTERN REGION,METROPOLITAN SOUTH EAST REGION,EASTERN REGION       0.00      0.00      0.00         0
                               METROPOLITAN NORTH WEST REGION       1.00      1.00      1.00      5013
METROPOLITAN NORTH WEST REGION,METROPOLITAN SOUTH EAST REGION       0.00      0.00      0.00         1
                               METROPOLITAN SOUTH EAST REGION       1.00      1.00      1.00      4925
METROPOLITAN SOUTH EAST REGION,METROPOLITAN NORTH WEST REGION       0.93      1.00      0.96        38
                                         NORTH EASTERN REGION       1.00      1.00      1.00       791
          NORTH EASTERN REGION,METROPOLITAN NORTH WEST REGION       0.00

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
print(red_X.columns)

MultiIndex([(                                  'DAY_OF_WEEK_1',),
            (                                  'DAY_OF_WEEK_2',),
            (                                  'DAY_OF_WEEK_3',),
            (                                  'DAY_OF_WEEK_4',),
            (                                  'DAY_OF_WEEK_5',),
            (                                  'DAY_OF_WEEK_6',),
            (                                  'DAY_OF_WEEK_7',),
            (    'ACCIDENT_TYPE_Collision with a fixed object',),
            (           'ACCIDENT_TYPE_Collision with vehicle',),
            (   'ACCIDENT_TYPE_Fall from or in moving vehicle',),
            ('ACCIDENT_TYPE_No collision and no object struck',),
            (                   'ACCIDENT_TYPE_Other accident',),
            (                'ACCIDENT_TYPE_Struck Pedestrian',),
            (                    'ACCIDENT_TYPE_Struck animal',),
            ('ACCIDENT_TYPE_Vehicle overturned (no collision)',),
          

In [40]:
red_unsupervised = crashes_df[['DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3', 'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5',
                                 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7', 'ACCIDENT_TYPE_Collision with a fixed object',
                                 'ACCIDENT_TYPE_Collision with vehicle', 'ACCIDENT_TYPE_Fall from or in moving vehicle',
                                 'ACCIDENT_TYPE_No collision and no object struck', 'ACCIDENT_TYPE_Other accident',
                                 'ACCIDENT_TYPE_Struck Pedestrian', 'ACCIDENT_TYPE_Struck animal',
                                 'ACCIDENT_TYPE_Vehicle overturned (no collision)',
                                 'ACCIDENT_TYPE_collision with some other object', 'LIGHT_CONDITION_Dark No street lights',
                                 'LIGHT_CONDITION_Dark Street lights off', 'LIGHT_CONDITION_Dark Street lights on',
                                 'LIGHT_CONDITION_Dark Street lights unknown', 'LIGHT_CONDITION_Day',
                                 'LIGHT_CONDITION_Dusk/Dawn', 'LIGHT_CONDITION_Unk.', 'ROAD_GEOMETRY_Cross intersection',
                                 'ROAD_GEOMETRY_Dead end', 'ROAD_GEOMETRY_Multiple intersection',
                                 'ROAD_GEOMETRY_Not at intersection', 'ROAD_GEOMETRY_Private property',
                                 'ROAD_GEOMETRY_Road closure', 'ROAD_GEOMETRY_T intersection', 'ROAD_GEOMETRY_Unknown',
                                 'ROAD_GEOMETRY_Y intersection','SPEED_ZONE_100 km/hr', 'SPEED_ZONE_110 km/hr',
                                 'SPEED_ZONE_40 km/hr', 'SPEED_ZONE_50 km/hr', 'SPEED_ZONE_60 km/hr', 'SPEED_ZONE_70 km/hr',
                                 'SPEED_ZONE_80 km/hr', 'SPEED_ZONE_90 km/hr', 'SPEED_ZONE_Camping grounds or off road',
                                 'SPEED_ZONE_Not known', 'SPEED_ZONE_Other speed limit', 'TOTAL_PERSONS', 'INJ_OR_FATAL',
                                 'MALES', 'FEMALES', 'UNLICENCSED', 'RMA_Arterial Highway', 'RMA_Arterial Other',
                                 'RMA_Freeway', 'RMA_Local Road', 'RMA_Non Arterial', 'RMA_ALL_Arterial Highway',
                                 'RMA_ALL_Arterial Highway,Arterial Other', 'RMA_ALL_Arterial Highway,Local Road',
                                 'RMA_ALL_Arterial Other', 'RMA_ALL_Arterial Other,Arterial Highway',
                                 'RMA_ALL_Arterial Other,Local Road', 'RMA_ALL_Freeway', 'RMA_ALL_Freeway,Arterial Other',
                                 'RMA_ALL_Local Road', 'RMA_ALL_Local Road,Arterial Highway',
                                 'RMA_ALL_Local Road,Arterial Other', 'RMA_ALL_Other', 'SEVERITY_Fatal accident',
                                 'SEVERITY_Non injury accident', 'SEVERITY_Other injury accident',
                                 'SEVERITY_Serious injury accident', 'REGION_NAME_ALL_EASTERN REGION,EASTERN REGION', 'REGION_NAME_ALL_METROPOLITAN NORTH WEST REGION', 'REGION_NAME_ALL_METROPOLITAN SOUTH EAST REGION', 'REGION_NAME_ALL_NORTH EASTERN REGION', 'REGION_NAME_ALL_NORTHERN REGION', 'REGION_NAME_ALL_Other', 'REGION_NAME_ALL_SOUTH WESTERN REGION', 'REGION_NAME_ALL_WESTERN REGION',]]

In [41]:
red_k_list = list(range(1, 11))
# Create an empty list to store the inertia values
red_inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in red_k_list:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(red_unsupervised)
    red_inertia.append(k_model.inertia_)

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 1

In [42]:
# Create a dictionary with the data to plot the Elbow curve
red_elbow_data = {"k": red_k_list, "inertia": red_inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_red_elbow = pd.DataFrame(red_elbow_data)

df_red_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=red_k_list
)

C:\Users\evang\anaconda3\envs\dev\lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
C:\Users\evang\anaconda3\envs\dev\lib\site-packages\holoviews\core\data\pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Curve   [k]   (inertia)